In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from cvxopt import matrix, solvers
from numpy import linalg
import cvxopt
import cvxopt.solvers
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix

In [5]:
class SVM(object):

    def __init__(self, kernel=linear_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: 
            self.C = float(self.C)


    def set_c (self,C):
        self.C = C
        if self.C is not None: 
            self.C = float(self.C)
    def set_kernel (self,kernel):
        self.kernel = kernel
        
    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self.kernel(X[i], X[j])

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        b = cvxopt.matrix(0.0)

        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))

        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

        # Lagrange multipliers
        a = np.ravel(solution['x'])

        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        ind = np.arange(len(a))[sv]
        self.__a = a[sv]
        self.__sv = X[sv]
        self.__sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.__a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.__a)):
            self.b += self.__sv_y[n]
            self.b -= np.sum(self.__a * self.__sv_y * K[ind[n],sv])
        self.b /= len(self.__a)

        # Weight vector
        if self.kernel == linear_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.__a)):
                self.w += self.__a[n] * self.__sv_y[n] * self.__sv[n]
        else:
            self.w = None

    def ــproject(self, X):
        if self.w is not None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.__a, self.__sv_y, self.__sv):
                    s += a * sv_y * self.kernel(X[i], sv)
                y_predict[i] = s
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.ــproject(X))
    

In [4]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=2):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
    return np.exp(-linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

In [6]:
train0 = np.array(pd.read_csv('train0.csv'))
train8 = np.array(pd.read_csv('train8.csv'))
train1 = np.array(pd.read_csv('train1.csv'))
train2 = np.array(pd.read_csv('train2.csv'))
X_train = []
X_train.append(train0)
X_train.append(train1)
X_train.append(train2)
X_train.append(train8)

# train_labels = np.concatenate((np.ones(train0.shape[0])*-1, (np.ones(train8.shape[0]))), axis=0)

In [7]:
test0 = pd.read_csv('test0.csv')
test8 = pd.read_csv('test8.csv')
test1 = pd.read_csv('test1.csv')
test2 = pd.read_csv('test2.csv')
X_test = []
X_test.append(test0)
X_test.append(test1)
X_test.append(test2)
X_test.append(test3)

# test_x = np.concatenate((test0,test8),axis = 0)
# test_labels = np.concatenate((np.ones(test0.shape[0])*-1 , (np.ones(test8.shape[0]))), axis=0)

In [8]:
del test0,test8,train0,train8

In [9]:
train_x = train_x/256
test_x = test_x/256

In [10]:
X, y = shuffle(train_x, train_labels, random_state=0)
X_test, y_test = shuffle(test_x, test_labels, random_state=0)

In [11]:
X_train, a, y_train, b = train_test_split(train_x, train_labels, test_size=0.8, random_state=42)

In [13]:
clf = SVM()
clf.fit(X_train, y_train)

y_predict = clf.predict(test_x)
correct = np.sum(y_predict == test_labels)
print("%d out of %d predictions correct" % (correct, len(y_predict)))
y_predict = clf.predict(test_x)
# confusion_matrix(y_predict,test_labels)
accuracy_score(y_predict,test_labels)

     pcost       dcost       gap    pres   dres
 0: -1.6092e+02 -5.0002e+02  1e+04  9e+01  3e+00
 1: -2.7509e+02 -4.0616e+02  6e+03  3e+01  1e+00
 2: -6.5023e+01 -6.4370e+01  1e+03  6e+00  2e-01
 3: -1.5506e+01 -2.3432e+01  3e+02  1e+00  4e-02
 4: -4.7927e+00 -1.2778e+01  7e+01  3e-01  1e-02
 5: -2.1500e+00 -9.6096e+00  3e+01  1e-01  3e-03
 6: -2.6882e+00 -6.5705e+00  7e+00  2e-02  7e-04
 7: -3.4906e+00 -5.5344e+00  3e+00  4e-03  1e-04
 8: -4.1817e+00 -5.2751e+00  1e+00  4e-16  5e-14
 9: -4.7065e+00 -5.0868e+00  4e-01  4e-16  5e-14
10: -4.9161e+00 -5.0318e+00  1e-01  4e-16  5e-14
11: -5.0075e+00 -5.0207e+00  1e-02  3e-16  6e-14
12: -5.0194e+00 -5.0197e+00  2e-04  7e-16  6e-14
13: -5.0196e+00 -5.0196e+00  3e-06  3e-16  6e-14
Optimal solution found.
125 support vectors out of 2354 points
1932 out of 1952 predictions correct


0.9897540983606558

In [16]:
clf = SVM(C = 0.1)
clf.fit(X_train, y_train)

y_predict = clf.predict(test_x)
correct = np.sum(y_predict == test_labels)
print("%d out of %d predictions correct" % (correct, len(y_predict)))
y_predict = clf.predict(test_x)
# confusion_matrix(y_predict,test_labels)
accuracy_score(y_predict,test_labels)

     pcost       dcost       gap    pres   dres
 0: -8.9733e+01 -4.9077e+02  2e+04  3e+01  3e-13
 1: -1.6046e+01 -4.5008e+02  1e+03  1e+00  3e-13
 2: -9.4998e+00 -1.7107e+02  3e+02  2e-01  6e-14
 3: -6.0286e+00 -8.8085e+01  1e+02  9e-02  3e-14
 4: -3.7287e+00 -3.9110e+01  5e+01  3e-02  2e-14
 5: -2.5182e+00 -1.7679e+01  2e+01  1e-02  2e-14
 6: -2.5923e+00 -8.2629e+00  7e+00  3e-03  2e-14
 7: -2.8186e+00 -4.4326e+00  2e+00  5e-16  2e-14
 8: -3.0719e+00 -3.9588e+00  9e-01  2e-16  2e-14
 9: -3.2407e+00 -3.6367e+00  4e-01  5e-16  2e-14
10: -3.3346e+00 -3.4535e+00  1e-01  2e-16  2e-14
11: -3.3698e+00 -3.4052e+00  4e-02  9e-16  2e-14
12: -3.3857e+00 -3.3868e+00  1e-03  9e-16  2e-14
13: -3.3862e+00 -3.3862e+00  2e-05  2e-16  2e-14
14: -3.3862e+00 -3.3862e+00  2e-07  2e-16  2e-14
Optimal solution found.
124 support vectors out of 2354 points
1937 out of 1952 predictions correct


0.9923155737704918

In [18]:
def gaussian_kernel(x, y, sigma= 2.0):
    return np.exp(-linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

clf = SVM(kernel = gaussian_kernel, C=0.1)
clf.fit(X_train, y_train)

y_predict = clf.predict(test_x)
correct = np.sum(y_predict == test_labels)
print("%d out of %d predictions correct" % (correct, len(y_predict)))
y_predict = clf.predict(test_x)
# confusion_matrix(y_predict,test_labels)
accuracy_score(y_predict,test_labels)

     pcost       dcost       gap    pres   dres
 0: -5.6046e+02 -5.4737e+02  8e+03  2e+01  1e-15
 1: -2.5119e+02 -4.3206e+02  5e+02  7e-01  1e-15
 2: -2.0184e+02 -2.6080e+02  6e+01  3e-15  2e-15
 3: -2.1217e+02 -2.2057e+02  8e+00  2e-15  2e-15
 4: -2.1209e+02 -2.1683e+02  5e+00  4e-15  1e-15
 5: -2.1192e+02 -2.1638e+02  4e+00  4e-15  1e-15
 6: -2.1228e+02 -2.1406e+02  2e+00  4e-16  1e-15
 7: -2.1261e+02 -2.1319e+02  6e-01  7e-15  1e-15
 8: -2.1279e+02 -2.1290e+02  1e-01  7e-15  2e-15
 9: -2.1283e+02 -2.1284e+02  1e-02  3e-15  2e-15
10: -2.1284e+02 -2.1284e+02  3e-04  4e-16  2e-15
11: -2.1284e+02 -2.1284e+02  4e-06  2e-15  2e-15
Optimal solution found.
2350 support vectors out of 2354 points
1652 out of 1952 predictions correct


0.8463114754098361

In [17]:
def gaussian_kernel(x, y, sigma= 10.0):
    return np.exp(-linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

clf = SVM(kernel = gaussian_kernel, C=0.1)
clf.fit(X_train, y_train)

y_predict = clf.predict(test_x)
correct = np.sum(y_predict == test_labels)
print("%d out of %d predictions correct" % (correct, len(y_predict)))
y_predict = clf.predict(test_x)
# confusion_matrix(y_predict,test_labels)
accuracy_score(y_predict,test_labels)

     pcost       dcost       gap    pres   dres
 0: -9.1288e+01 -5.0600e+02  2e+04  3e+01  3e-15
 1: -3.6075e+01 -4.6719e+02  1e+03  1e+00  3e-15
 2: -2.1870e+01 -1.6932e+02  1e+02  4e-15  2e-15
 3: -3.0465e+01 -6.4597e+01  3e+01  7e-16  2e-15
 4: -3.4931e+01 -4.9218e+01  1e+01  2e-16  2e-15
 5: -3.6656e+01 -4.4694e+01  8e+00  2e-15  2e-15
 6: -3.7944e+01 -4.1598e+01  4e+00  2e-16  2e-15
 7: -3.8650e+01 -4.0117e+01  1e+00  6e-16  2e-15
 8: -3.9010e+01 -3.9443e+01  4e-01  2e-16  2e-15
 9: -3.9153e+01 -3.9209e+01  6e-02  5e-16  2e-15
10: -3.9176e+01 -3.9178e+01  2e-03  2e-15  2e-15
11: -3.9177e+01 -3.9177e+01  5e-05  3e-16  2e-15
12: -3.9177e+01 -3.9177e+01  9e-07  2e-15  2e-15
Optimal solution found.
616 support vectors out of 2354 points
1936 out of 1952 predictions correct


0.9918032786885246

In [14]:
clf = SVM(kernel = polynomial_kernel ,C=0.1)
clf.fit(X_train, y_train)

y_predict = clf.predict(test_x)
correct = np.sum(y_predict == test_labels)
print("%d out of %d predictions correct" % (correct, len(y_predict)))
y_predict = clf.predict(test_x)
# confusion_matrix(y_predict,test_labels)
accuracy_score(y_predict,test_labels)

     pcost       dcost       gap    pres   dres
 0: -6.7477e-02 -2.3585e+02  5e+03  1e+01  6e-14
 1: -6.3526e-02 -1.9553e+02  2e+02  2e-01  9e-14
 2: -3.5599e-02 -2.5714e+01  3e+01  2e-02  9e-14
 3: -3.6711e-02 -5.0704e+00  6e+00  3e-03  7e-14
 4: -3.0102e-02 -1.8992e+00  2e+00  1e-03  3e-14
 5: -2.3610e-02 -1.0317e+00  1e+00  6e-04  2e-14
 6: -1.3822e-02 -3.2390e-01  4e-01  2e-04  8e-15
 7: -1.1167e-02 -6.8246e-02  7e-02  2e-05  6e-15
 8: -1.2075e-02 -2.3446e-02  1e-02  3e-06  6e-15
 9: -1.2826e-02 -1.6760e-02  4e-03  2e-16  6e-15
10: -1.3854e-02 -1.4832e-02  1e-03  1e-16  6e-15
11: -1.4165e-02 -1.4325e-02  2e-04  2e-16  6e-15
12: -1.4226e-02 -1.4236e-02  1e-05  2e-16  6e-15
13: -1.4230e-02 -1.4230e-02  2e-07  2e-16  6e-15
14: -1.4230e-02 -1.4230e-02  3e-09  2e-16  6e-15
Optimal solution found.
171 support vectors out of 2354 points
1942 out of 1952 predictions correct


0.9948770491803278